In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import os

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [11]:
train_data = torch.utils.data.DataLoader(datasets.MNIST('data', train=True, download=True, 
                        transform=transforms.Compose([transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))])),
                        batch_size=64, shuffle=True)

In [12]:
class PytorchDrawer(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 5, 1, 1)
        self.conv2 = nn.Conv2d(32, 64, 5, 1, 1)
        self.conv3 = nn.Conv2d(64, 128, 5, 1, 1)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = F.max_pool2d(F.relu(self.conv3(x)), 2)
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [13]:
model = PytorchDrawer().to(device)
optimizer = optim.Adam(model.parameters())

def train(epoch):
    model.train()
    for batch_id, (data, target) in enumerate(train_data):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        out = model(data)
        loss = F.nll_loss(out, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
            epoch, batch_id * len(data), len(train_data.dataset),
                    100. * batch_id / len(train_data), loss.data))

for epoch in range(1, 20):
    train(epoch)

Die letzten 5000 Zeilen der Streamingausgabe wurden abgeschnitten.
Train Epoch: 14 [40192/60000 (67%)]	Loss: 0.000114
Train Epoch: 14 [40256/60000 (67%)]	Loss: 0.000769
Train Epoch: 14 [40320/60000 (67%)]	Loss: 0.027012
Train Epoch: 14 [40384/60000 (67%)]	Loss: 0.000394
Train Epoch: 14 [40448/60000 (67%)]	Loss: 0.001057
Train Epoch: 14 [40512/60000 (67%)]	Loss: 0.000016
Train Epoch: 14 [40576/60000 (68%)]	Loss: 0.000053
Train Epoch: 14 [40640/60000 (68%)]	Loss: 0.000142
Train Epoch: 14 [40704/60000 (68%)]	Loss: 0.000008
Train Epoch: 14 [40768/60000 (68%)]	Loss: 0.170842
Train Epoch: 14 [40832/60000 (68%)]	Loss: 0.157789
Train Epoch: 14 [40896/60000 (68%)]	Loss: 0.008293
Train Epoch: 14 [40960/60000 (68%)]	Loss: 0.000087
Train Epoch: 14 [41024/60000 (68%)]	Loss: 0.000099
Train Epoch: 14 [41088/60000 (68%)]	Loss: 0.000016
Train Epoch: 14 [41152/60000 (69%)]	Loss: 0.000351
Train Epoch: 14 [41216/60000 (69%)]	Loss: 0.000098
Train Epoch: 14 [41280/60000 (69%)]	Loss: 0.000620
Train Epoch: 14

In [15]:
model.eval()
device = torch.device("cpu")
model.to(device)

save = input('save? y ')
if save == 'y':
    model.eval()
    model_folder_path = './model'
    file_name='Pytorch.pth'
    if not os.path.exists(model_folder_path):
        os.makedirs(model_folder_path)
    file_name = os.path.join(model_folder_path, file_name)
    torch.save(model, file_name)

save? y y
